# Spotify Analysis - Logistic Regression Model

### We'll fit a logistic regression model to Spotify data, to create a model to predict a songs success. In this model, a song is considered successful if it received more than the average streams of the dataset. A success was denoted with a "1", otherwise it received a "0". This metric was saved into a new column, "song_success" ("y"). The initial features ("X") of the model include various metrics such as danceability, liveliness, energy, etc. After we were able to successfully make predictions with our model, we added more features to the model to improve upon its predictive accuracy. These other metrics were in spotify charts, in spotify playlists, and bpm. 

In [2]:
# Import neccessary libraries
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report

In [6]:
# Uploaded csv file with Spotify data, then constructed a dataframe from the 
# csv file
file_path = Path(r"C:/Users/ppate/dm_spotify.csv")
original_df = pd.read_csv(file_path)
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,11/29/2019,Blinding Lights,The Weeknd,1,2019,11,29,43899,69,3703895074,...,171,C#,Major,50,38,80,0,0,9,7
1,1/6/2017,Shape of You,Ed Sheeran,1,2017,1,6,32181,10,3562543890,...,96,C#,Minor,83,93,65,58,0,9,8
2,11/8/2018,Someone You Loved,Lewis Capaldi,1,2018,11,8,17836,53,2887241814,...,110,C#,Major,50,45,41,75,0,11,3
3,5/10/2019,Dance Monkey,Tones and I,1,2019,5,10,24529,0,2864791672,...,98,F#,Minor,82,54,59,69,0,18,10
4,10/9/2018,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee",2,2018,10,9,24094,78,2808096550,...,90,D,Major,76,91,50,54,0,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,7/14/2023,Overdrive,Post Malone,1,2023,7,14,410,36,14780425,...,140,C#,Major,56,48,73,0,0,35,4
948,6/1/2023,"Gol Bolinha, Gol Quadrado 2","Mc Pedrinho, DJ 900",2,2023,6,1,293,8,11956641,...,133,B,Minor,93,68,65,42,0,12,25
949,7/13/2023,QUEMA,"Sog, Ryan Castro, Peso Pluma",3,2023,7,13,437,31,11599388,...,97,NaN,Major,79,92,89,5,0,6,5
950,12/22/2022,Jhoome Jo Pathaan,"Arijit Singh, Vishal Dadlani, Sukriti Kakar, V...",6,2022,12,22,138,4,1365184,...,105,G,Major,82,62,74,10,0,33,7


In [7]:
# Sort the dataframe by the "Date" column from least recent to most recent
original_df.sort_values('Date', ascending=True, inplace=True)
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
822,1/1/1930,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,90598517,...,130,F#,Minor,65,49,80,22,4,7,5
373,1/1/1942,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",3,1942,1,1,11940,0,395591396,...,96,A,Major,23,19,25,91,0,40,3
323,1/1/1950,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",2,1950,1,1,10585,0,473248298,...,143,D,Major,60,86,32,88,0,34,6
372,1/1/1952,A Holly Jolly Christmas - Single Version,Burl Ives,1,1952,1,1,7930,0,395591396,...,140,NaN,Major,67,81,36,64,0,15,3
209,1/1/1957,Jingle Bell Rock,Bobby Helms,1,1957,1,1,10326,0,741301563,...,119,D,Major,74,78,37,84,0,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,9/9/2016,Say You Won't Let Go,James Arthur,1,2016,9,9,15722,16,2420461338,...,99,A#,Major,40,45,56,69,0,9,5
95,9/9/2021,Shivers,Ed Sheeran,1,2021,9,9,10147,30,1302184087,...,141,D,Major,79,82,86,28,0,4,9
361,9/9/2021,Angel Baby,Troye Sivan,1,2021,9,9,1959,9,408843328,...,145,B,Major,56,41,57,1,0,13,3
544,9/9/2022,Forget Me,Lewis Capaldi,1,2022,9,9,2520,4,239411309,...,102,C#,Minor,67,72,74,30,0,36,4


In [8]:
# Rename dataframe columns to remove unneccessary characters, referencing these
# columns will be easier moving forward 
original_df = original_df.rename(columns={'danceability_%':'danceability',
                                              'valence_%':'valence',
                                              'energy_%':'energy',
                                              'acousticness_%':'acousticness',
                                              'instrumentalness_%':'instrumentalness',
                                              'liveness_%':'liveness',
                                              'speechiness_%':'speechiness'
                                              })
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
822,1/1/1930,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,90598517,...,130,F#,Minor,65,49,80,22,4,7,5
373,1/1/1942,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",3,1942,1,1,11940,0,395591396,...,96,A,Major,23,19,25,91,0,40,3
323,1/1/1950,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",2,1950,1,1,10585,0,473248298,...,143,D,Major,60,86,32,88,0,34,6
372,1/1/1952,A Holly Jolly Christmas - Single Version,Burl Ives,1,1952,1,1,7930,0,395591396,...,140,NaN,Major,67,81,36,64,0,15,3
209,1/1/1957,Jingle Bell Rock,Bobby Helms,1,1957,1,1,10326,0,741301563,...,119,D,Major,74,78,37,84,0,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,9/9/2016,Say You Won't Let Go,James Arthur,1,2016,9,9,15722,16,2420461338,...,99,A#,Major,40,45,56,69,0,9,5
95,9/9/2021,Shivers,Ed Sheeran,1,2021,9,9,10147,30,1302184087,...,141,D,Major,79,82,86,28,0,4,9
361,9/9/2021,Angel Baby,Troye Sivan,1,2021,9,9,1959,9,408843328,...,145,B,Major,56,41,57,1,0,13,3
544,9/9/2022,Forget Me,Lewis Capaldi,1,2022,9,9,2520,4,239411309,...,102,C#,Minor,67,72,74,30,0,36,4


In [9]:
# Check column names of the dataframe, in the next cell we will drop unncessary
# columns
#original_df.columns

In [10]:
# Drop several uneeded columns, these columns will not be used as features in X
# for the preliminary model
original_df.drop(["released_year", "released_month", "released_day", 
                  "in_deezer_playlists", "in_deezer_charts",
                  "in_shazam_charts", "key", 'mode',
                  "in_apple_playlists", "in_apple_charts"],
                 axis = 1)
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
822,1/1/1930,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,90598517,...,130,F#,Minor,65,49,80,22,4,7,5
373,1/1/1942,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",3,1942,1,1,11940,0,395591396,...,96,A,Major,23,19,25,91,0,40,3
323,1/1/1950,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",2,1950,1,1,10585,0,473248298,...,143,D,Major,60,86,32,88,0,34,6
372,1/1/1952,A Holly Jolly Christmas - Single Version,Burl Ives,1,1952,1,1,7930,0,395591396,...,140,NaN,Major,67,81,36,64,0,15,3
209,1/1/1957,Jingle Bell Rock,Bobby Helms,1,1957,1,1,10326,0,741301563,...,119,D,Major,74,78,37,84,0,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,9/9/2016,Say You Won't Let Go,James Arthur,1,2016,9,9,15722,16,2420461338,...,99,A#,Major,40,45,56,69,0,9,5
95,9/9/2021,Shivers,Ed Sheeran,1,2021,9,9,10147,30,1302184087,...,141,D,Major,79,82,86,28,0,4,9
361,9/9/2021,Angel Baby,Troye Sivan,1,2021,9,9,1959,9,408843328,...,145,B,Major,56,41,57,1,0,13,3
544,9/9/2022,Forget Me,Lewis Capaldi,1,2022,9,9,2520,4,239411309,...,102,C#,Minor,67,72,74,30,0,36,4


In [11]:
# Create an empty column in the dataframe to hold the to-be-calculated value

original_df["song_success"] = np.nan
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness,song_success
822,1/1/1930,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,90598517,...,F#,Minor,65,49,80,22,4,7,5,NaN
373,1/1/1942,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",3,1942,1,1,11940,0,395591396,...,A,Major,23,19,25,91,0,40,3,NaN
323,1/1/1950,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",2,1950,1,1,10585,0,473248298,...,D,Major,60,86,32,88,0,34,6,NaN
372,1/1/1952,A Holly Jolly Christmas - Single Version,Burl Ives,1,1952,1,1,7930,0,395591396,...,NaN,Major,67,81,36,64,0,15,3,NaN
209,1/1/1957,Jingle Bell Rock,Bobby Helms,1,1957,1,1,10326,0,741301563,...,D,Major,74,78,37,84,0,6,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,9/9/2016,Say You Won't Let Go,James Arthur,1,2016,9,9,15722,16,2420461338,...,A#,Major,40,45,56,69,0,9,5,NaN
95,9/9/2021,Shivers,Ed Sheeran,1,2021,9,9,10147,30,1302184087,...,D,Major,79,82,86,28,0,4,9,NaN
361,9/9/2021,Angel Baby,Troye Sivan,1,2021,9,9,1959,9,408843328,...,B,Major,56,41,57,1,0,13,3,NaN
544,9/9/2022,Forget Me,Lewis Capaldi,1,2022,9,9,2520,4,239411309,...,C#,Minor,67,72,74,30,0,36,4,NaN


In [12]:
# Calculate the average number of streams for each song in the dataframe, and 
# assign it to the variable "average_streams"

average_streams = original_df["streams"].mean()
average_streams_in_millions = original_df["streamsM"].mean()
print(f"The average amount of streams amongst the song in the dataframe is roughly {int(average_streams_in_millions)} million, {int(average_streams)} is the exact number.")

The average amount of streams amongst the song in the dataframe is roughly 514 million, 514137424 is the exact number.


In [26]:
# Determined which songs are successful by comparing the number of streams for
# the song to "average_streams". These values were inserted in the 
#"song_success" column. A successful song is denoted by a "1". 

original_df['song_success'] = (original_df['streams'] > average_streams).astype(int) 
original_df

,Date,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness,song_success
822,1/1/1930,Agudo Mï¿½ï¿½gi,"Styrx, utku INC, Thezth",3,1930,1,1,323,0,90598517,...,F#,Minor,65,49,80,22,4,7,5,0
373,1/1/1942,White Christmas,"Bing Crosby, John Scott Trotter & His Orchestr...",3,1942,1,1,11940,0,395591396,...,A,Major,23,19,25,91,0,40,3,0
323,1/1/1950,Let It Snow! Let It Snow! Let It Snow!,"Frank Sinatra, B. Swanson Quartet",2,1950,1,1,10585,0,473248298,...,D,Major,60,86,32,88,0,34,6,0
372,1/1/1952,A Holly Jolly Christmas - Single Version,Burl Ives,1,1952,1,1,7930,0,395591396,...,NaN,Major,67,81,36,64,0,15,3,0
209,1/1/1957,Jingle Bell Rock,Bobby Helms,1,1957,1,1,10326,0,741301563,...,D,Major,74,78,37,84,0,6,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,9/9/2016,Say You Won't Let Go,James Arthur,1,2016,9,9,15722,16,2420461338,...,A#,Major,40,45,56,69,0,9,5,1
95,9/9/2021,Shivers,Ed Sheeran,1,2021,9,9,10147,30,1302184087,...,D,Major,79,82,86,28,0,4,9,1
361,9/9/2021,Angel Baby,Troye Sivan,1,2021,9,9,1959,9,408843328,...,B,Major,56,41,57,1,0,13,3,0
544,9/9/2022,Forget Me,Lewis Capaldi,1,2022,9,9,2520,4,239411309,...,C#,Minor,67,72,74,30,0,36,4,0


In [27]:
# Build a logistic regression model. 
# Seperate the data into labels and features. X will be the features, and y will
# be the label.

X = original_df[["danceability", "valence", "energy", "acousticness", "liveness", "instrumentalness", "speechiness"]]
y = original_df["song_success"]

In [28]:
# Review the X variable Dataframe
X

,danceability,valence,energy,acousticness,liveness,instrumentalness,speechiness
822,65,49,80,22,7,4,5
373,23,19,25,91,40,0,3
323,60,86,32,88,34,0,6
372,67,81,36,64,15,0,3
209,74,78,37,84,6,0,3
...,...,...,...,...,...,...,...
14,40,45,56,69,9,0,5
95,79,82,86,28,4,0,9
361,56,41,57,1,13,0,3
544,67,72,74,30,36,0,4


In [29]:
# Review the y variable Series
y

822    0
373    0
323    0
372    0
209    1
      ..
14     1
95     1
361    0
544    0
655    0
Name: song_success, Length: 952, dtype: int32

In [30]:
# Check the balance of our target values
y.value_counts()

song_success
0    651
1    301
Name: count, dtype: int64

In [31]:
# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2,
                                                    random_state=1)

In [32]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
regression_model = LogisticRegression(random_state = 1)

# Fit the model using training data
regression_model.fit(X_train, y_train)

# Make a prediction using the testing data
y_predictions = regression_model.predict(X_test)

C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

In [33]:
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.71      0.99      0.82       135
           1       0.33      0.02      0.03        56

    accuracy                           0.70       191
   macro avg       0.52      0.50      0.43       191
weighted avg       0.60      0.70      0.59       191



C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

## The initial logistic regression model was successfully able to make predictions with 70% accuracy as seen in the classification report. 

In [34]:
# Build a second logistic regression model with additional features. 
# Seperate the data into labels and features. X will be the features, and y 
# will be the label.

X2 = original_df[["danceability", "valence", "energy", "acousticness", "liveness", "instrumentalness", "speechiness","bpm","in_spotify_charts","in_spotify_playlists", "artist_count"]]
y2 = original_df["song_success"]

In [35]:
# Review the X2 variable Dataframe
X2

,danceability,valence,energy,acousticness,liveness,instrumentalness,speechiness,bpm,in_spotify_charts,in_spotify_playlists,artist_count
822,65,49,80,22,7,4,5,130,0,323,3
373,23,19,25,91,40,0,3,96,0,11940,3
323,60,86,32,88,34,0,6,143,0,10585,2
372,67,81,36,64,15,0,3,140,0,7930,1
209,74,78,37,84,6,0,3,119,0,10326,1
...,...,...,...,...,...,...,...,...,...,...,...
14,40,45,56,69,9,0,5,99,16,15722,1
95,79,82,86,28,4,0,9,141,30,10147,1
361,56,41,57,1,13,0,3,145,9,1959,1
544,67,72,74,30,36,0,4,102,4,2520,1


In [36]:
# Review the y2 variable Series
y2

822    0
373    0
323    0
372    0
209    1
      ..
14     1
95     1
361    0
544    0
655    0
Name: song_success, Length: 952, dtype: int32

In [37]:
# Split the data using train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, 
                                                    y2, 
                                                    test_size = 0.2,
                                                    random_state=1)

In [38]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
regression_model2 = LogisticRegression(random_state = 1)

# Fit the model using training data
regression_model2.fit(X2_train, y2_train)

# Make a prediction using the testing data
y2_predictions = regression_model2.predict(X2_test)

C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

In [47]:
print(classification_report(y2_test, y2_predictions))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90       135
           1       0.79      0.68      0.73        56

    accuracy                           0.85       191
   macro avg       0.83      0.80      0.82       191
weighted avg       0.85      0.85      0.85       191



C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

## After including the in_spotify_playlists, in_spotify_charts, and bpm columns as features in "X", the accuracy of models predictions increased by 15%

In [48]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y2_test, y2_predictions)

C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\ppate\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

0.8022486772486772